In [118]:
import numpy as np
import pandas as pd

2. What is the average age of the players?

In [119]:
match_away_team_df=pd.read_csv('MatchAwayTeamInfo.csv')
match_home_team_df=pd.read_csv('MatchHomeTeamInfo.csv')
height_player_mean=pd.concat([match_away_team_df.height,match_home_team_df.height]).mean()

4. What is the longest match recorded in terms of duration?

In [120]:
match_time_df=pd.read_csv('MatchTimeInfo.csv')
match_time_df.head()
match_time_df.fillna(0,inplace=True)
match_time_df.drop_duplicates(inplace=True)

In [121]:

match_time_df['sumation_time_periods_match']=match_time_df.iloc[:,[1,2,3,4,5]].sum(axis=1)
match_time_df.head()

,match_id,period_1,period_2,period_3,period_4,period_5,current_period_start_timestamp,sumation_time_periods_match
0,12260075,3463.0,3855.0,0.0,0.0,0.0,1.714511e+09,7318.0
1,12260076,3032.0,2121.0,2674.0,0.0,0.0,1.714492e+09,7827.0
2,12260077,2747.0,3525.0,4074.0,0.0,0.0,1.714492e+09,10346.0
3,12260078,2519.0,2531.0,2121.0,0.0,0.0,1.714578e+09,7171.0
4,12260080,2616.0,2766.0,0.0,0.0,0.0,1.714483e+09,5382.0


In [122]:
longest_time=match_time_df['sumation_time_periods_match'].max()
longest_match=match_time_df.loc[(match_time_df['sumation_time_periods_match']==longest_time),:]
longest_match

,match_id,period_1,period_2,period_3,period_4,period_5,current_period_start_timestamp,sumation_time_periods_match
13350,12346747,167761.0,3392.0,0.0,0.0,0.0,1.716382e+09,171153.0


6. Which country has produced the most successful tennis players?

In [138]:
a=pd.concat([match_away_team_df[['full_name','country','current_rank']],match_home_team_df[['full_name','country','current_rank']]])
a.sort_values('current_rank').drop_duplicates().dropna().iloc[0:19,:]

,full_name,country,current_rank
13359,"Swiatek, Iga",Poland,1.0
11242,"Djokovic, Novak",Serbia,1.0
11529,"Sabalenka, Aryna",Belarus,2.0
11035,"Sinner, Jannik",Italy,2.0
6170,"Gauff, Cori",USA,3.0
10767,"Alcaraz, Carlos",Spain,3.0
4,"Zverev, Alexander",Germany,4.0
499,"Rybakina, Elena",Kazakhstan,4.0
4589,"Medvedev, Daniil",Russia,4.0
4115,"Zverev, Alexander",Germany,5.0


In [139]:
a=pd.concat([match_away_team_df[['full_name','country','total_prize']],match_home_team_df[['full_name','country','total_prize']]])
a.sort_values('total_prize',ascending=False).drop_duplicates().iloc[0:19,:]

,full_name,country,total_prize
14368,"Djokovic, Novak",Serbia,151703255.0
11242,"Djokovic, Novak",Serbia,151675883.0
4844,"Djokovic, Novak",Serbia,151629548.0
10733,"Nadal, Rafael",Spain,112340134.0
4578,"Nadal, Rafael",Spain,112313000.0
10710,"Murray, Andy",United Kingdom,53781808.0
7104,"Murray, Andy",United Kingdom,53778542.0
7090,"Zverev, Alexander",Germany,34897595.0
5599,"Zverev, Alexander",Germany,34020353.0
12081,"Medvedev, Daniil",Russia,33950097.0
